In [11]:
import pandas as pd
import numpy as np
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import torch
from torch.utils.data import Dataset
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.metrics import roc_auc_score
from clockwork_helperfunc import *
from clockwork_helperfunc import evaluation
import clockwork_helperfunc 
from imp import reload  
from tqdm import tqdm
reload(clockwork_helperfunc)
import time

#configuration
batch_size = 5
num_epochs = 50
number_units = 7

### Data

In [2]:
training = pd.read_pickle('/Users/leilei/Documents/DS1005/CW/truncate_train_data.p')
val = pd.read_pickle('/Users/leilei/Documents/DS1005/CW/truncate_valid_data.p')

In [3]:
patient_feature_train, label_train, label_time_train = downsample(training, proportional = 2)

In [4]:
sum(label_time_train), sum(label_train*label_time_train)

(529367, 232732)

In [5]:
patient_feature_val = val[0] #before cleaning, original data
label_val = val[1]
label_time_val = val[2]

In [6]:
Counter(label_train), Counter(label_val)

(Counter({0: 3535, 1: 1790}), Counter({0: 4089, 1: 511}))

In [7]:
len(patient_feature_train)

5325

In [8]:
set(label_train[:3535])

{0}

### Model

In [9]:
### Model
#forward
#cell_class, step
class LSTM_Model(nn.Module):
    def __init__(self, number_units, batch_size, mean = 0, std = 0.1, input_dim = 40):
        super(LSTM_Model, self).__init__()
        '''
        scale: the updating frequency, a list. [1,2,4,8,16,32]
        batch_size: the size of batch
        group_size: the number of nodes in each scale, default is 1.
        mean: the mean of Gaussian distribution for initialize weights for hidden layer
        std: the standard devation of the Gaussian distribution for initialize weights for hidden layer
        input_dim: the feature dimension of each time step
        '''
        self.hidden_dim = number_units
        self.input_dim = input_dim
        self.batch_size = batch_size

        self.lstm = nn.LSTM(self.input_dim, self.hidden_dim)
        self.hidden2tag = nn.Linear(self.hidden_dim, 2)
        
    def forward(self, sequence, hidden):#depends on what passed for model.train(), to be filled)
        '''
        sequence: batch  x timestep x number_feature matrix
        hidden: should be h0
        ''' 
        sequence = sequence.transpose(0,1) #now sequence, batch, num_feature
        #print(sequence.size())
        #print(hidden[0].size())
        lstm_out, hidden = self.lstm(sequence, hidden) #hidden is the last one of sequence
        #print('lstm_size')
        #print(lstm_out.size())
        #print(hidden)
        tag_space = self.hidden2tag(lstm_out.view(len(sequence)*self.batch_size, -1))
        #previously last step
        #self.hidden2tag(hidden[0])
        #final_score = F.log_softmax(self.hidden2tag(hidden[0]))
        #print(final_score[0])
        tag_scores = F.log_softmax(tag_space)       
        return tag_scores#, final_score[0]#it is batch * class, the first dimension is 1, so remove it

    def init_hidden(self):
        return (Variable(torch.zeros(1, self.batch_size, self.hidden_dim)),
                Variable(torch.zeros(1, self.batch_size, self.hidden_dim)))


In [ ]:
lstm = LSTM_Model(number_units, batch_size)
loss = torch.nn.CrossEntropyLoss(ignore_index=-1)  
optimizer = torch.optim.Adam(lstm.parameters(), lr=0.005)
accuracy_list = []
train_loader, validation_loader = reload_data(batch_size,patient_feature_train, label_train, label_time_train,patient_feature_val, label_val, label_time_val)
start = time.time()
for epoch in tqdm(range(num_epochs)):
    for step, (data, label,label_list) in enumerate(train_loader):        
        data, label = Variable(data), Variable(label)
        lstm.zero_grad()
        hidden= lstm.init_hidden()
        output = lstm(data, hidden)
        
        label = label.transpose(0,1).contiguous().view(-1) 
        #output = torch.stack(output, dim=1).view(-1, 2) 
        #print(output.size())
        #print(label.size())
        lossy = loss(output, label)
        lossy.backward()
        optimizer.step()
        #print('one_step')
        if step % 100 ==0:
            print("Epoch: {}; Loss: {}".format(epoch, lossy.data[0]))
                #print('accuracy_on_training: {}'.format(evaluation(train_loader))) 
            acc0, acc1, val_acc, auc = evaluation(validation_loader, lstm,'lstm')
            print('accuracy_on_validation: {}, the acc for live is {}, the acc for dead is {}'.format(val_acc, acc0, acc1)) 
            print('auc ' +str(auc))
    accuracy_list.append(val_acc)
    if ((epoch > 5) and ((accuracy_list[-1] < (accuracy_list[-2] - 0.01)) or (accuracy_list[-1] < (accuracy_list[-3] - 0.01)))):
        print("early stop, accuracy = ", accuracy_list[-2])
        break     
end = time.time()

  0%|          | 0/50 [00:00<?, ?it/s]

Epoch: 0; Loss: 0.6911388635635376


In [ ]:
print(end - start)

In [88]:
### Model
#forward
#cell_class, step
class GRU_Model(nn.Module):
    def __init__(self, number_units, batch_size, mean = 0, std = 0.1, input_dim = 48):
        super(GRU_Model, self).__init__()
        '''
        scale: the updating frequency, a list. [1,2,4,8,16,32]
        batch_size: the size of batch
        group_size: the number of nodes in each scale, default is 1.
        mean: the mean of Gaussian distribution for initialize weights for hidden layer
        std: the standard devation of the Gaussian distribution for initialize weights for hidden layer
        input_dim: the feature dimension of each time step
        '''
        self.hidden_dim = number_units
        self.input_dim = input_dim
        self.batch_size = batch_size

        self.gru = nn.GRU(self.input_dim, self.hidden_dim)
        self.hidden2tag = nn.Linear(self.hidden_dim, 2)
        
    def forward(self, sequence, hidden):#depends on what passed for model.train(), to be filled)
        '''
        sequence: batch  x timestep x number_feature matrix
        hidden: should be h0
        ''' 
        sequence = sequence.transpose(0,1)
        gru_out, hidden = self.gru(sequence, hidden)
        tag_space = self.hidden2tag(gru_out.view(len(sequence)*self.batch_size, -1))
        #self.hidden2tag(hidden[0])
        #final_score = F.log_softmax(self.hidden2tag(hidden[0]))
        tag_scores = F.log_softmax(tag_space)       
        return tag_scores#, final_score[0]
    
    def init_hidden(self):
        return Variable(torch.zeros(1, self.batch_size, self.hidden_dim))

In [12]:
gru = GRU_Model(number_units, batch_size)
loss = torch.nn.CrossEntropyLoss(ignore_index=-1)  
optimizer = torch.optim.Adam(gru.parameters(), lr=0.05)
accuracy_list = []
train_loader, validation_loader = reload_data(batch_size)

for epoch in range(num_epochs):
    for step, (data, label) in enumerate(train_loader):        
        data, label = Variable(data), Variable(label)
        gru.zero_grad()
        hidden= gru.init_hidden()
        output = gru(data, hidden)
        
        label = label.transpose(0,1).contiguous().view(-1) 
        lossy = loss(output, label)
        lossy.backward()
        optimizer.step()
        #print('one_step')
        
        if step % 100 ==0:
            print("Epoch: {}; Loss: {}".format(epoch, lossy.data[0]))
                #print('accuracy_on_training: {}'.format(evaluation(train_loader))) 
            acc0, acc1, val_acc, auc = evaluation(validation_loader, lstm,'lstm')
            print('accuracy_on_validation: {}, the acc for live is {}, the acc for dead is {}'.format(val_acc, acc0, acc1)) 
            print('auc ' +str(auc))
    accuracy_list.append(val_acc)
    if ((epoch > 5) and ((accuracy_list[-1] < (accuracy_list[-2] - 0.01)) or (accuracy_list[-1] < (accuracy_list[-3] - 0.01)))):
        print("early stop, accuracy = ", accuracy_list[-2])
        break         

Epoch: 0; Loss: 0.6647575497627258
<class 'list'> <class 'list'> <class 'list'> <class 'list'>
accuracy_on_validation: 0.5551138934982652, the acc for live is 0.3667269451784597, the acc for dead is 0.7868698720294042
Epoch: 1; Loss: 0.6899204254150391
<class 'list'> <class 'list'> <class 'list'> <class 'list'>
accuracy_on_validation: 0.5857802836023533, the acc for live is 0.8131206607163002, the acc for dead is 0.3061033100774216
Epoch: 2; Loss: 0.6666716933250427
<class 'list'> <class 'list'> <class 'list'> <class 'list'>
accuracy_on_validation: 0.6146345602655001, the acc for live is 0.7729232115570658, the acc for dead is 0.419905882946646
Epoch: 3; Loss: 0.6320374011993408
<class 'list'> <class 'list'> <class 'list'> <class 'list'>
accuracy_on_validation: 0.5793294614572334, the acc for live is 0.955269697912358, the acc for dead is 0.116843083574093
Epoch: 4; Loss: 0.7337309122085571
<class 'list'> <class 'list'> <class 'list'> <class 'list'>
accuracy_on_validation: 0.5993833911